In [ ]:
!python -m spacy download en
!pip install transformers
!pip install torch
!pip install tensorflow
!python -m spacy download en_core_web_lg

In [1]:
import numpy as np
import re
import networkx as nx
import spacy
from summa import keywords
from summa.summarizer import summarize

# Laden des Spacy-Modells
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
import torch

In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print ("MPS device not found.")

In [29]:
#model_name='facebook/bart-large-cnn'
#model_name='facebook/bart-large'
#model_name='NICFRU/bart-base-paraphrasing'
#model_name='NICFRU/bart-base-paraphrasing-science'
#model_name='NICFRU/bart-base-paraphrasing-news'
#model_name='NICFRU/bart-base-paraphrasing-story'
model_name='NICFRU/bart-base-paraphrasing-review'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model_name)

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [3]:
def reduce_repetitions(text):
    # Wir benutzen reguläre Ausdrücke (regex), um Wiederholungen von Zeichen zu reduzieren.
    text = re.sub(r'\.{2,}', '.', text)  # Ersetzt zwei oder mehr Punkte durch einen Punkt.
    text = re.sub(r'\!{2,}', '!', text)  # Ersetzt zwei oder mehr Ausrufezeichen durch ein Ausrufezeichen.
    text = re.sub(r'\,{2,}', ',', text)  # Ersetzt zwei oder mehr Kommas durch ein Komma.
    text = re.sub(r'\;{2,}', ';', text)  # Ersetzt zwei oder mehr Semikolons durch ein Semikolon.
    return text  # Gibt den bereinigten Text zurück.

def textrank_extractive(text, compression_rate=0.5,split='\. '):
    # Hier verwenden wir Spacy, um den Text in Sätze zu zerlegen und zu tokenisieren.
    nlp = spacy.load("en_core_web_lg")
    doc = re.split(fr'(?<!\b\w\w){split}', reduce_repetitions(re.sub(' +', ' ', text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))))
    sentences = [sent for sent in doc if len(sent.replace("-", " ").split()) > 2]
    sentence_docs = [nlp(sentence) for sentence in sentences]

    # Hier verwenden wir TextRank, um wichtige Sätze zu extrahieren.
    num_sentences = max(1, int(len(sentences) * compression_rate))
    extracted_sentences = summarize(text, words=num_sentences, split=True)

    # Wir bauen eine Matrix auf, die die Ähnlichkeit zwischen den Sätzen misst.
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i, doc_i in enumerate(sentence_docs):
        for j, doc_j in enumerate(sentence_docs):
            similarity = similarity_function(doc_i, doc_j)
            similarity_matrix[i, j] = similarity

    # Hier erstellen wir einen Graphen, in dem die Sätze die Knoten und die Ähnlichkeiten die Kanten sind.
    graph = nx.from_numpy_array(similarity_matrix)

    # Nun berechnen wir den TextRank-Score für jeden Satz.
    scores = nx.pagerank_numpy(graph)

    # Wir wählen die besten Sätze basierend auf ihren TextRank-Scores aus.
    top_sentences = sorted(scores, key=scores.get, reverse=True)[:num_sentences]

    # Die ausgewählten Sätze werden nach ihrer Position im Text sortiert.
    top_sentences = sorted(top_sentences)

    # Schließlich geben wir die extrahierten Schlüsselsätze zurück.
    extracted_sentences = [sentences[index] for index in top_sentences]
    return extracted_sentences

def similarity_function(doc1, doc2):
    # Wir berechnen die Cosinus-Ähnlichkeit zwischen den beiden Dokumenten.
    similarity = doc1.similarity(doc2)
    return similarity

def compression_ratio(text, summary):
    # Wir berechnen das Verhältnis der Anzahl der Wörter in der Zusammenfassung zur Anzahl der Wörter im Ausgangstext.
    num_words_text = len(text.split())
    num_words_summary = len(summary.split())
    ratio = num_words_summary / num_words_text
    return ratio

def compression(text, compression_rate,split='\. '):
    max_iterations = 20
    iterations = 0
    extracted = textrank_extractive(text, compression_rate,split)
    summary = '. '.join(extracted)
    compression_rate_renwed = compression_rate

    # Wenn das Kompressionsverhältnis kleiner ist als die gewünschte Rate, versuchen wir, das Kompressionsverhältnis zu erhöhen.
    while compression_ratio(text, summary) < compression_rate and iterations < max_iterations:
        iterations += 1
        compression_rate_renwed += 0.05
        if compression_rate_renwed > 1:
            compression_rate_renwed = 1
        extracted = textrank_extractive(text, compression_rate=compression_rate_renwed)
        summary = '. '.join(extracted)
    return summary  # Gibt die komprimierte Zusammenfassung zurück.


In [4]:
def token_count(text):
    # Wir zerlegen den Text in einzelne Worte (Token) und zählen diese.
    tokens = text.split()
    return len(tokens)  # Gibt die Anzahl der Wörter (Token) im Text zurück.

def adjust_length(text):
    # Zuerst zählen wir die Anzahl der Wörter (Token) im Text.
    length = token_count(text)
    
    # Wir passen die Mindest- und Maxmimallänge des Textes basierend auf der aktuellen Länge an.
    # Die genauen Werte sind hier variabel und hängen von der Länge des Textes ab.
    
    if length <20:
        # Wenn der Text weniger als 20 Wörter hat, erhöhen wir die Mindestlänge um 5% der aktuellen Länge.
        # Die maximale Länge wird dann auf das Doppelte der Mindestlänge gesetzt.
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length
    elif length <50:
        # Wenn der Text zwischen 20 und 50 Wörtern hat, verwenden wir die gleichen Regeln wie zuvor, 
        # aber die maximale Länge wird auf das 1,5-fache der Mindestlänge gesetzt.
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length* 0.5
    elif length <60:
        # Bei Texten zwischen 50 und 60 Wörtern erhöhen wir die Mindestlänge um 5% der aktuellen Länge.
        # Die maximale Länge wird dann auf das 1,4-fache der Mindestlänge gesetzt.
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length* 0.4
    elif length < 80:
        # Bei Texten zwischen 60 und 80 Wörtern erhöhen wir die Mindestlänge um 5% der aktuellen Länge.
        # Die maximale Länge wird dann auf das 1,25-fache der Mindestlänge gesetzt.
        min_length = length + int(length * 0.05)
        max_length = min_length + min_length* 0.25
    elif length < 100:
        # Bei Texten zwischen 80 und 100 Wörtern erhöhen wir die Mindestlänge um 30% der aktuellen Länge.
        # Die maximale Länge wird dann auf die Mindestlänge plus 100 Wörter gesetzt.
        min_length = length + int(length * 0.3)
        max_length = min_length + 100
    else:
        # Bei Texten mit 100 Wörtern oder mehr setzen wir die Mindestlänge auf das nächste Vielfache von 70,
        # das größer als die aktuelle Länge geteilt durch 50 ist. 
        # Die maximale Länge wird dann auf die Mindestlänge plus 100 Wörter gesetzt.
        min_length = math.ceil(length / 50) * 70
        max_length = min_length + 100

    # Wir geben die berechneten minimalen und maximalen Längen zurück.
    return min_length, max_length


In [5]:
def batch_sent(sentenc,splitt=180,split='\. '):
    # Teile den eingegebenen Text in einzelne Sätze auf, wobei jeder Satz durch ". " getrennt ist.
    # Hierbei wird sichergestellt, dass das ". " nicht auf ein einzelnes Wort folgt.
    sentences = re.split(fr'(?<!\b\w\w){split}', sentenc.lower())

    # Initialisierung der Batches und der aktuellen Batch-Liste sowie der aktuellen Batch-Länge.
    batches = []
    batch = []
    batch_len = 0
    
    # Durchlaufen Sie jeden Satz in den Sätzen.
    for sentence in sentences:
        # Berechnen Sie die Anzahl der Tokens im Satz.
        sentence_len = len(tokenizer.tokenize(sentence))
        
        # Wenn die Hinzufügung des aktuellen Satzes die maximale Batch-Länge überschreitet...
        if sentence_len + batch_len > splitt:
            # ...und wenn der aktuelle Satz weniger als die maximale Batch-Länge hat...
            if sentence_len < splitt:  
                # ...füge die aktuelle Batch-Liste zu den Batches hinzu...
                batches.append(batch)
                # ...und beginne eine neue Batch-Liste mit dem aktuellen Satz.
                batch = [sentence]
                # Die aktuelle Batch-Länge wird auf die Länge des aktuellen Satzes gesetzt.
                batch_len = sentence_len
            # Sätze, die länger als die maximale Batch-Länge sind, werden übersprungen.
        else:
            # Wenn der aktuelle Satz zur aktuellen Batch-Liste hinzugefügt werden kann, ohne die maximale Batch-Länge zu überschreiten...
            # ...füge den Satz zur aktuellen Batch-Liste hinzu...
            batch.append(sentence)
            # ...und erhöhe die aktuelle Batch-Länge um die Länge des aktuellen Satzes.
            batch_len += sentence_len
    
    # Füge die letzte Batch-Liste zu den Batches hinzu.
    batches.append(batch)

    # Die Funktion gibt die erstellten Batches zurück.
    return batches


In [6]:
def paraphrase_of_text(df_s, text_name='text', komp_name='reduction_multiplier', split='\. '):
    # Erstellen eines leeren DataFrame, in das die generierten Zusammenfassungen und deren Details eingefügt werden.
    df_summary_testing = pd.DataFrame(columns=['Zusammenfassung','Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate','länge Zusammenfassung','länge Ausgangstext','batch_texts','batch_output'])

    # Durchlaufen Sie jede Zeile im gegebenen DataFrame.
    for _, row in tqdm(df_s.iterrows(), total=df_s.shape[0]):
        batch_text_list=[]
        output_text_list=[]
        text = row[text_name]
        komp = row[komp_name]
        text_gesamt_list=[]

        # Teilen Sie den Text in kleinere Batches auf.
        for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
            if len(batch):
                # Fügen Sie die Sätze in einem Batch zusammen.
                batch_text = '. '.join(batch)
                batch_text += "."
                batch_text_list.append(batch_text)
                # Passen Sie die Länge des Textes an.
                min_length_test, max_length_test = adjust_length(batch_text)
                # Paraphrasieren Sie den Text.
                ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*komp,0)), min_length=int(round(min_length_test*komp,0)),length_penalty=100,num_beams=2)

                # Speichern Sie die generierte Zusammenfassung.
                text_gesamt_list.append(ext_summary[0]['generated_text'])

        # Fügen Sie alle generierten Zusammenfassungen zusammen.
        text_gesamt = '. '.join(text_gesamt_list)

        # Berechnen Sie die tatsächliche Kompressionsrate.
        actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100

        # Erstellen Sie einen neuen DataFrame für die aktuellen Ergebnisse.
        df_current = pd.DataFrame({
            'Zusammenfassung': [text_gesamt],
            'Min_Kompressionsrate': [min_length_test],
            'Max_Kompressionsrate': [max_length_test],
            'Endgueltige_Kompressionsrate': [actual_compression_rate],
            'länge Zusammenfassung': [len(text_gesamt.split(' '))],
            'länge Ausgangstext': [len(text.split(' '))],
            'batch_texts': [batch_text_list],
            'batch_output': [text_gesamt_list]
        })

        # Fügen Sie die Daten zum DataFrame hinzu.
        df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)

    return df_summary_testing


In [7]:
def calculate_compression(df, total_tokens_col, current_tokens_col, desired_compression_rate):
    # Berechnung der aktuellen Kompressionsrate als Verhältnis von aktueller Tokenanzahl zu Gesamt-Tokenanzahl.
    df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
    # Berechnung der Differenz zwischen der gewünschten und der aktuellen Kompressionsrate.
    df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
    # Berechnung des Reduktionsmultiplikators als Verhältnis der gewünschten zur aktuellen Kompressionsrate.
    df['reduction_multiplier'] = df[desired_compression_rate] / df['current_compression_rate']
    # Rückgabe des DataFrame mit den berechneten Kompressionsinformationen.
    return df


In [8]:
def text_rank_algo(df, seed=10, split='\\. ', random_T=True, column='text'):
    # Initialisierung des Rückgabe-DataFrames mit den angegebenen Spalten
    df_return = pd.DataFrame(columns=['text', 'text_rank_text', 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate', 'text_rank_compression_rate'])
    
    # Festlegen des Seeds für den Zufallsgenerator
    random.seed(seed)
    
    # Iteration über die Zeilen des gegebenen DataFrame
    for index, row in tqdm(df.iterrows()):
        text = row[column].replace("\n", " ")
        
        # Entscheidung über den zu verwendenden Kompressionsfaktor
        if random_T:
            random_value = round(random.uniform(0.2, 0.8), 2)  # Generieren eines Zufallswertes zwischen 0.2 und 0.8
        else:
            if row['reduction_multiplier'] < 0.8:
                random_value = row['desired_compression_rate']
            elif row['reduction_multiplier'] < 0.9:
                random_value = row['reduction_multiplier']
            else:
                random_value = 1

        # Anwendung der Text Rank Kompression
        text_rank_text = compression(text.replace("\n\n", " "), random_value, split)
        
        # Berechnung des Kompressionsverhältnisses
        compression_ratio_value = compression_ratio(text, compression(text, random_value, split))
        
        # Bereinigung des Textes und des text_rank_text von mehrfachen Leerzeichen und bestimmten Zeichen
        text = re.sub(' +', ' ', text.replace("\n", " ").replace('-', ' ').replace('_', ' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))
        text_rank_text = re.sub(' +', ' ', text_rank_text.replace("\n", " ").replace('-', ' ').replace('_', ' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))
        
        # Erstellung eines DataFrame mit den aktuellen Ergebnissen
        df_current = pd.DataFrame({
            'text': [text],
            'text_rank_text': [text_rank_text],
            'tokens_gesamt': [len(text.split(' '))],
            'token_text_rank': [len(text_rank_text.split(' '))],
            'desired_compression_rate': [random_value],
            'text_rank_compression_rate': [compression_ratio_value]
        })

        # Hinzufügen der aktuellen Ergebnisse zum Rückgabe-DataFrame
        df_return = pd.concat([df_return, df_current], ignore_index=True)

    # Rückgabe des erstellten DataFrames mit den Kompressionsinformationen
    return df_return


In [9]:
def execute_text_gen(df, split='\\. ', seed=10):
    # Anwendung des Text Rank Algorithmus auf das gegebene DataFrame
    rank_df = text_rank_algo(df, seed=seed, split=split)
    
    # Berechnung der Kompressionsrate basierend auf dem resultierenden DataFrame des Text Rank Algorithmus
    df_zwischen = calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
    
    # Erzeugen der Paraphrasen des Texts basierend auf dem DataFrame, das durch die Berechnung der Kompressionsrate erhalten wurde
    df_sum = paraphrase_of_text(df_zwischen[['text_rank_text','reduction_multiplier']], text_name='text_rank_text', split=split)
    
    # Zusammenführen der beiden DataFrames zu einem einzigen DataFrame
    merged_df = pd.concat([df_sum, df_zwischen], axis=1)
    
    # Berechnung der endgültigen Kompressionsrate
    merged_df['ent_com_rate'] = merged_df['länge Zusammenfassung'] / merged_df['tokens_gesamt']
    
    # Erneute Berechnung der Kompressionsrate basierend auf dem zusammengeführten DataFrame
    new_df = calculate_compression(merged_df[['Zusammenfassung','länge Zusammenfassung','text','tokens_gesamt','desired_compression_rate','ent_com_rate']], 'tokens_gesamt', 'länge Zusammenfassung', 'desired_compression_rate')
    
    # Erneute Anwendung des Text Rank Algorithmus auf das aktualisierte DataFrame, um verlängerte texte zu reduzieren
    df_testen = text_rank_algo(new_df, random_T=False, column='Zusammenfassung')
    
    # Zusammenführen der Daten und Berechnung der endgültigen Kompressionsrate
    a = pd.concat([df_testen[['text_rank_text','token_text_rank',]], new_df[['Zusammenfassung','länge Zusammenfassung','text','tokens_gesamt','desired_compression_rate','ent_com_rate']]], axis=1)
    a['ent_com_rate'] = a['länge Zusammenfassung'] / a['tokens_gesamt']
    
    # Rückgabe des finalen DataFrame
    return a


In [10]:
df_test=pd.read_csv('data/data_test.csv')

In [16]:
df_test.classification.unique()

array(['Scientific', 'news', 'reviews', 'story'], dtype=object)

In [ ]:
df_training=df_test[df_test.classification=='story'].reset_index(drop=True)[['classification','text']].reset_index(drop=True).copy()
rank_df=text_rank_algo(df_training,seed=10,split='\. ',random_T=True,column='text')
df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
df_zwischen.to_csv(f'data/ergbnisse/test_story_text_rank.csv')

In [ ]:
execute_text_gen(df_test[df_test.classification=='reviews'].reset_index(drop=True)[['classification','text']].reset_index(drop=True).copy(),split='\. ',seed=10).to_csv(f'data/ergbnisse/test_review_text.csv')

In [41]:
test_dict=df_test[['classification', 'text']][0:1].to_dict('records')[0]
